In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import scipy.signal as ss
import sys

import tools.data_reader_sfi as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

In [ ]:
# FEATURE EXTRACTION - HF RR, LF RR
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
is_clean_ecg = False
data_type = dr.Signals.ECG
measure = "hf_rr"
# measure = "lf_rr"
fs = 100.0
# f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]


subjects = dr.SUBJECTS
phases = "all"

out = []

for subject in subjects:
    try:
        signals = dr.get_signal(subject, phases="all", signal=dr.Signals.ECG)
    except Exception as e:
        print(f"{e} " + "-"*40)
        continue
    columns = ["subject"] + list(signals.keys())
    subject_metrics = [subject]

    for clip in signals.keys():
        signal = signals[clip]
        if measure == "hf_rr":
            value = preprocessing.get_hf_rr(signal)
        else:
            value = preprocessing.get_lf_rr(signal)
        subject_metrics.append(np.mean(value))
    out.append(subject_metrics)
out = pd.DataFrame(data=out, columns=columns)
print(out.head())
file = os.path.join(dr.Paths.METRICS, f"{measure}.csv")
# with open(file, "w+") as f:
#     out.to_csv(f)
    


In [ ]:
# FEATURE EXTRACTION WITH HEARTPY - BPM, RMSSD, IBI
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp


convert_sr = False
is_clean_ecg = False
data_type = dr.Signals.ECG
measures = ["bpm", "rmssd", "ibi", "sdnn", "breathingrate"]
# measure = "bpm"
# measure = "rmssd"
# measure = "ibi"
# measure = "sdnn"
# measure = "breathingrate"
fs = 100.0


subjects = dr.SUBJECTS
phases = "all"

out = []

for measure in measures:
    for subject in subjects:
        print(f"SUBJECT {subject} " + "-"*40)
        try:
            signals = dr.get_signal(subject, phases="all", signal=dr.Signals.ECG)
        except Exception as e:
            print(f"{e} " + "-"*40)
            continue
        columns = ["subject"] + list(signals.keys())
        subject_metrics = [subject]

        for clip in signals.keys():
            signal = signals[clip].to_numpy().flatten()
            working_data, measures = hp.process_segmentwise(signal, fs, segment_width=55, segment_overlap=5/55)
            value = measures[f"{measure}"]
            subject_metrics.append(np.mean(value))
        out.append(subject_metrics)
    out = pd.DataFrame(data=out, columns=columns)
    file = os.path.join(dr.Paths.METRICS, f"{measure}.csv")
    with open(file, "w+") as f:
                out.to_csv(f)


In [54]:
# FEATURE EXTRACTION - EDA
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


convert_sr = False
is_clean_ecg = False
data_type = dr.Signals.EDA
measure = "mean_SCL"
# measure = "SCR_rate"
fs = 100.0


subjects = dr.SUBJECTS
phases = "all"

out = []

for measure in measures:
    for subject in subjects:
        print(f"SUBJECT {subject} " + "-"*40)
        try:
            signals = dr.get_signal(subject, phases="all", signal=dr.Signals.EDA)
        except Exception as e:
            print(f"{e} " + "-"*40)
            continue
        columns = ["subject"] + list(signals.keys())
        subject_metrics = [subject]

        for clip in signals.keys():
            signal = signals[clip].to_numpy().flatten()
            if measure == "mean_SCL":
                value = preprocessing.get_mean_SCL(signal, fs)
            else:
                value = preprocessing.get_SCR_rate(signal, fs)
            subject_metrics.append(np.mean(value))
        out.append(subject_metrics)
    out = pd.DataFrame(data=out, columns=columns)
    file = os.path.join(dr.Paths.METRICS, f"{measure}.csv")
    with open(file, "w+") as f:
                out.to_csv(f)


SUBJECT 2 ----------------------------------------
[]
[23 23]


c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


[]
[25 23]
[]
[22 20]
[]
[19 17]
[21 19]
[]
[20 18]
[20 18]
[17 16]
[17 16]
[]
[]
[14 14 14 14 14 15 14 14 15 14 13 13 13 14 14 15 15 17 17 17 16 18 19 21
 22 20 22 22 22 19 19 19 20 21 22 21 21 22 22 23 23 24 26 27 27 26 27 26
 26 24]
SUBJECT 3 ----------------------------------------


KeyboardInterrupt: 